# Peephole LSTM Test & Performance Comparison (Speed & Memory)

* [Imports](#Importing-necessary-modules)
* [Load & Definition](#Loading-and-defining-modules)
    * [Autograd Functions](#Autograd-Functions)
    * [Module Classes](#Module-classes-(C++,-CUDA,-PyTorch))
* [Models](#Defining-models)
    * [Definition](#Definition)
    * [Instantiation](#Instantiation)
    * [Parameter Synchronization](#Parameter-Synchronization)
* [Fake Dataset](#Creating-a-fake-dataset)
* [Sanity Check](#Sanity-check:-output-comparison)
    * [Forward Outputs](#Forward-Outputs)
    * [Backward Gradients](#Backward-Gradients)
* [Forward Performance](#Forward-time-comparison)
* [+Backward Performance](#+Backward-time-comparison)

---

## Importing necessary modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [1]:
import torch
from torch import nn
from torch.utils.cpp_extension import load
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import math
from collections import OrderedDict
from time import sleep

torch.__version__

'0.4.1'

---

## Loading and defining modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Autograd Functions
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [2]:
_bn_peephole_lstm_layer_cpp = load('bn_peephole_lstm_layer', ['./bn_peephole_lstm_layer.cpp'])

########################################################################################################################

class BNPeepholeLSTMFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias, gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell, beta_tanh_cell, running_mean_ih, running_mean_hh, running_mean_ch, running_mean_tanh_cell, running_var_ih, running_var_hh, running_var_ch, running_var_tanh_cell, old_h, old_cell, momentum, epsilon, dropout_p, training):
        
        outputs = _bn_peephole_lstm_layer_cpp.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                    gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell, beta_tanh_cell,
                                                    running_mean_ih, running_mean_hh, running_mean_ch, running_mean_tanh_cell,
                                                    running_var_ih, running_var_hh, running_var_ch, running_var_tanh_cell,
                                                    old_h, old_cell,
                                                    momentum, epsilon, dropout_p, training)
        
        (out, new_h, new_cell,
         running_mean_ih.data, running_mean_hh.data, running_mean_ch.data, running_mean_tanh_cell.data,
         running_var_ih.data, running_var_hh.data, running_var_ch.data, running_var_tanh_cell.data) = outputs[:11]
        
        variables = outputs[11:] + [weight_ih, weight_hh, weight_ch,
                                    gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell]
        ctx.training = training # Boolean value stored this way since only tensors can be stored using save_for_backward
        ctx.save_for_backward(*variables)

        return out, new_h, new_cell

    @staticmethod
    def backward(ctx, grad_output, grad_h, grad_cell):
        outputs = _bn_peephole_lstm_layer_cpp.backward(
            grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors, ctx.training)
        
        (d_old_h, d_old_cell, d_input,
         d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
         d_gamma_ih, d_gamma_hh, d_gamma_ch, d_gamma_tanh_cell, d_beta_tanh_cell) = outputs
        
        return d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias, d_gamma_ih, d_gamma_hh, d_gamma_ch, d_gamma_tanh_cell, d_beta_tanh_cell, None, None, None, None, None, None, None, None, d_old_h, d_old_cell, None, None, None, None
   

c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:92: UserWarning: Error checking compiler version: Command '['c++']' returned non-zero exit status 1.
  warnings.warn('Error checking compiler version: {}'.format(error))
c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:118: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) may be ABI-incompatible with PyTorch!
Please use a compiler that is ABI-compatible with GCC 4.9 and above.
See https://gcc.gnu.org/onlinedocs/libstdc++/manual/abi.html.

See https://gist.github.com/goldsborough/d466f43e8ffc948ff92de7486c5216d6
for instructions on how to install GCC 4.9 or higher.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(ABI_INCOMPATIBILITY_WARNING.format(compiler))


### Module classes (C++, CUDA, PyTorch)
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [3]:
class BNPeepholeLSTMTorch(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0., momentum=0.1, eps=1e-05):
        if not 0 <= dropout <= 1:
            raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
        if not 0 <= momentum <= 1:
            raise ValueError(f"Invalid momentum value : {momentum} momentum must be in range [0, 1].")
        super(BNPeepholeLSTMTorch, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = float(dropout)
        self.momentum = float(momentum)
        self.eps = eps
        
        self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
        self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
        self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size, hidden_size)))
        self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))
        
        self.register_parameter('gamma_ih', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_hh', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_ch', nn.Parameter(torch.empty(3 * hidden_size)))
        self.register_parameter('gamma_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('beta_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        
        self.register_buffer('running_mean_ih', torch.empty(4 * hidden_size))
        self.register_buffer('running_mean_hh', torch.empty(4 * hidden_size))
        self.register_buffer('running_mean_ch', torch.empty(3 * hidden_size))
        self.register_buffer('running_mean_tanh_cell', torch.empty(hidden_size))
        self.register_buffer('running_var_ih', torch.empty(4 * hidden_size))
        self.register_buffer('running_var_hh', torch.empty(4 * hidden_size))
        self.register_buffer('running_var_ch', torch.empty(3 * hidden_size))
        self.register_buffer('running_var_tanh_cell', torch.empty(hidden_size))
        
        self.reset_parameters()
        self.reset_running_stats()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
        self.weight_ih.data.uniform_(-stdv, +stdv)
        self.weight_hh.data.uniform_(-stdv, +stdv)
        self.weight_ch.data.uniform_(-stdv, +stdv)
        
        self.bias.data.zero_()
        self.bias.data[:self.hidden_size].fill_(1.)
        
        self.gamma_ih.data.uniform_()
        self.gamma_hh.data.uniform_()
        self.gamma_ch.data.uniform_()
        self.gamma_tanh_cell.data.uniform_()
        self.beta_tanh_cell.data.zero_()
    
    def reset_running_stats(self):
        self.running_mean_ih.data.zero_()
        self.running_mean_hh.data.zero_()
        self.running_mean_ch.data.zero_()
        self.running_mean_tanh_cell.data.zero_()
        self.running_var_ih.data.fill_(1.)
        self.running_var_hh.data.fill_(1.)
        self.running_var_ch.data.fill_(1.)
        self.running_var_tanh_cell.data.fill_(1.)
    
    def forward(self, input, states):
        assert input.dim() == 3
        outputs = input.new_empty((input.size(0), input.size(1), self.hidden_size))
        
        h = states[0].clone()
        c = states[1].clone()
        
        weight_ih = self.weight_ih.t()
        weight_hh = self.weight_hh.t()
        weight_ch = self.weight_ch.t()
        
        ih = torch.matmul(input.transpose(0, 1), weight_ih)
        
        for i in range(input.size(1)):
            h = F.dropout(h, p=self.dropout, training=self.training)
            
            gates = (F.batch_norm(ih[i], self.running_mean_ih, self.running_var_ih, weight=self.gamma_ih, bias=None, training=self.training, momentum=self.momentum, eps=self.eps)
                     + F.batch_norm(torch.mm(h, weight_hh), self.running_mean_hh, self.running_var_hh, weight=self.gamma_hh, bias=None, training=self.training, momentum=self.momentum, eps=self.eps)
                     + self.bias)
            
            gates = torch.cat((gates[:, :3 * self.hidden_size].add(F.batch_norm(torch.mm(c, weight_ch), self.running_mean_ch, self.running_var_ch, weight=self.gamma_ch, bias=None, training=self.training, momentum=self.momentum, eps=self.eps)).sigmoid(), gates[:, 3 * self.hidden_size:].tanh()), dim=1).chunk(chunks=4, dim=1)
            
            c = torch.addcmul(gates[1] * gates[3], c, gates[0])
            h = gates[2] * F.batch_norm(c.tanh(), self.running_mean_tanh_cell, self.running_var_tanh_cell, weight=self.gamma_tanh_cell, bias=self.beta_tanh_cell, training=self.training, momentum=self.momentum, eps=self.eps)
            
            outputs[:, i] = h
        
        outputs = torch.nn.functional.dropout(outputs, p=self.dropout, training=self.training)
    
        return outputs, (h, c)
    
    def __repr__(self):
        return f"BNPeepholeLSTMTorch(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout}, momentum={self.momentum}, eps={self.eps})"

########################################################################################################################

class BNPeepholeLSTMCPP(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0., momentum=0.1, eps=1e-05):
        if not 0 <= dropout <= 1:
            raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
        if not 0 <= momentum <= 1:
            raise ValueError(f"Invalid momentum value : {momentum} momentum must be in range [0, 1].")
            
        super(BNPeepholeLSTMCPP, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = float(dropout)
        self.momentum = float(momentum)
        self.eps = eps
        
        self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
        self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
        self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size, hidden_size)))
        self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))
        
        self.register_parameter('gamma_ih', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_hh', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_ch', nn.Parameter(torch.empty(3 * hidden_size)))
        self.register_parameter('gamma_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('beta_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        
        self.register_buffer('running_mean_ih', torch.empty(4 * hidden_size))
        self.register_buffer('running_mean_hh', torch.empty(4 * hidden_size))
        self.register_buffer('running_mean_ch', torch.empty(3 * hidden_size))
        self.register_buffer('running_mean_tanh_cell', torch.empty(hidden_size))
        self.register_buffer('running_var_ih', torch.empty(4 * hidden_size))
        self.register_buffer('running_var_hh', torch.empty(4 * hidden_size))
        self.register_buffer('running_var_ch', torch.empty(3 * hidden_size))
        self.register_buffer('running_var_tanh_cell', torch.empty(hidden_size))
        
        self.reset_parameters()
        self.reset_running_stats()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
        self.weight_ih.data.uniform_(-stdv, +stdv)
        self.weight_hh.data.uniform_(-stdv, +stdv)
        self.weight_ch.data.uniform_(-stdv, +stdv)
        
        self.bias.data.zero_()
        self.bias.data[:self.hidden_size].fill_(1.)
        
        self.gamma_ih.data.uniform_()
        self.gamma_hh.data.uniform_()
        self.gamma_ch.data.uniform_()
        self.gamma_tanh_cell.data.uniform_()
        self.beta_tanh_cell.data.zero_()
    
    def reset_running_stats(self):
        self.running_mean_ih.data.zero_()
        self.running_mean_hh.data.zero_()
        self.running_mean_ch.data.zero_()
        self.running_mean_tanh_cell.data.zero_()
        self.running_var_ih.data.fill_(1.)
        self.running_var_hh.data.fill_(1.)
        self.running_var_ch.data.fill_(1.)
        self.running_var_tanh_cell.data.fill_(1.)
        
    def forward(self, input, state):
        input = input.transpose(0, 1).contiguous()
        
        output, new_h, new_cell = BNPeepholeLSTMFunction.apply(input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias, self.gamma_ih, self.gamma_hh, self.gamma_ch, self.gamma_tanh_cell, self.beta_tanh_cell, self.running_mean_ih, self.running_mean_hh, self.running_mean_ch, self.running_mean_tanh_cell, self.running_var_ih, self.running_var_hh, self.running_var_ch, self.running_var_tanh_cell, state[0], state[1], self.momentum, self.eps, self.dropout, self.training)
        
        return output.transpose(0, 1).contiguous(), (new_h, new_cell)
    
    def __repr__(self):
        return f"BNPeepholeLSTMCPP(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout}, momentum={self.momentum}, eps={self.eps})"

---

## Defining models
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Definition
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [4]:
class BNPeepholeTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0, momentum=0.1, eps=1e-05):
        super().__init__()
        self.lstm0 = BNPeepholeLSTMTorch(input_size, hidden_size, dropout, momentum, eps)
        self.lstm1 = BNPeepholeLSTMTorch(hidden_size, hidden_size, dropout, momentum, eps)
        self.lstm2 = BNPeepholeLSTMTorch(hidden_size, hidden_size, dropout, momentum, eps)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states

########################################################################################################################
    
class BNPeepholeCPP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0, momentum=0.1, eps=1e-05):
        super().__init__()
        self.lstm0 = BNPeepholeLSTMCPP(input_size, hidden_size, dropout, momentum, eps)
        self.lstm1 = BNPeepholeLSTMCPP(hidden_size, hidden_size, dropout, momentum, eps)
        self.lstm2 = BNPeepholeLSTMCPP(hidden_size, hidden_size, dropout, momentum, eps)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states

### Instantiation
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [5]:
device = ('cpu', 'cuda')[1]

input_size = 5
hidden_size = 20
output_size = 10
dropout = 0.
momentum = 0.1
eps = 1e-05

model_torch = BNPeepholeTorch(input_size, hidden_size, output_size, dropout, momentum, eps)
model_cpp = BNPeepholeCPP(input_size, hidden_size, output_size, dropout, momentum, eps)

model_torch.to(device)
model_cpp.to(device)

models = (model_torch, model_cpp)

### Parameter Synchronization
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [6]:
named_parameter_dicts = [
    dict(model_torch.named_parameters()),
    dict(model_cpp.named_parameters())
]

print("Synchronized Parameters:\n")
for common_param_name in set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
    print("\t{}".format(common_param_name))
    for i in range(1, len(named_parameter_dicts)):
        if named_parameter_dicts[i][common_param_name].size() == named_parameter_dicts[0][common_param_name].size():
            named_parameter_dicts[i][common_param_name].data = named_parameter_dicts[0][common_param_name].data
        else:
            raise RuntimeError("Size mismatch\n0:{}\n{i}:{}".format(named_parameter_dicts[0][common_param_name].size(),
                                                                    named_parameter_dicts[i][common_param_name].size()))
print()
print("Exclusive Parameters (Not Synchronized):\n")
for exclusive_param_name in set.union(*(set(npd.keys()) for npd in named_parameter_dicts)) - set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
    print("\t{}".format(exclusive_param_name))

Synchronized Parameters:

	lstm0.gamma_tanh_cell
	lstm1.weight_ih
	lstm1.gamma_ch
	lstm2.beta_tanh_cell
	lstm1.beta_tanh_cell
	fc.bias
	lstm0.weight_hh
	lstm1.bias
	lstm1.weight_hh
	lstm2.gamma_tanh_cell
	lstm1.gamma_hh
	lstm0.gamma_ch
	lstm2.gamma_ih
	lstm2.weight_ch
	lstm2.gamma_ch
	lstm1.weight_ch
	lstm1.gamma_ih
	lstm2.gamma_hh
	lstm2.weight_hh
	lstm0.beta_tanh_cell
	lstm0.gamma_hh
	lstm1.gamma_tanh_cell
	lstm0.weight_ch
	lstm0.gamma_ih
	lstm0.weight_ih
	lstm2.bias
	fc.weight
	lstm0.bias
	lstm2.weight_ih

Exclusive Parameters (Not Synchronized):



---

## Creating a fake dataset
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [7]:
def create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True):
    fake_inputs = torch.randn(dataset_size, sequence_length, input_size)
    fake_targets = torch.randint(high=output_size, size=(dataset_size, sequence_length), dtype=torch.int64)

    fake_dataset = TensorDataset(fake_inputs, fake_targets)

    fake_loader = DataLoader(fake_dataset, batch_size=batch_size, drop_last=drop_last)
    
    return fake_loader

In [8]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size)
print(next(iter(fake_loader))[0].size(), next(iter(fake_loader))[1].size())

torch.Size([32, 20, 5]) torch.Size([32, 20])


---

## Sanity check: output comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Forward Outputs
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [9]:
hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

inputs = next(iter(fake_loader))[0].to(device)

for model in models:
    model.train()
#     model.eval()

with torch.no_grad():
    print("[model_torch]")
    print("\n{partial output}")
    out0 = model_torch(inputs, hidden)[0]
    print(out0[:4, -2:, :7])
    print("\n{partial running stat}")
    print(model_torch.lstm0.running_mean_hh[:16])
    print(model_torch.lstm0.running_var_hh[:16])
    print("\n")
    print("[model_cpp]")
    print("\n{partial output}")
    out1 = model_cpp(inputs, hidden)[0]
    print(out1[:4, -2:, :7])
    print("\n{partial running stat}")
    print(model_cpp.lstm0.running_mean_hh[:16])
    print(model_cpp.lstm0.running_var_hh[:16])
    print()
    print(out0.sub(out1).abs().sum())

[model_torch]

{partial output}
tensor([[[ 0.1182, -0.1024, -0.0847,  0.4884, -0.3113, -0.1015, -0.4723],
         [ 0.1088, -0.0691, -0.0678,  0.5072, -0.2716, -0.0872, -0.5118]],

        [[ 0.7502, -0.1577, -0.0714, -0.1778,  0.1666,  0.1662,  0.2979],
         [ 0.6923, -0.1450, -0.0882, -0.0734,  0.0916,  0.1482,  0.2066]],

        [[ 0.0834, -0.0708,  0.1137,  0.5564, -0.1223, -0.2362, -0.5450],
         [ 0.0620, -0.0354,  0.1041,  0.5649, -0.1621, -0.2186, -0.5781]],

        [[ 0.1797, -0.1557,  0.1224,  0.4256, -0.1555, -0.2707, -0.2062],
         [ 0.1927, -0.1282,  0.1652,  0.4314, -0.0964, -0.3142, -0.2469]]],
       device='cuda:0')

{partial running stat}
tensor([-0.0036,  0.0050,  0.0042, -0.0065,  0.0267, -0.0185, -0.0233, -0.0079,
         0.0011, -0.0031,  0.0053, -0.0020, -0.0084,  0.0053, -0.0012, -0.0055],
       device='cuda:0')
tensor([0.1303, 0.1333, 0.1307, 0.1311, 0.1347, 0.1334, 0.1297, 0.1307, 0.1301,
        0.1240, 0.1294, 0.1267, 0.1269, 0.1254, 0.1344,

### Backward Gradients
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [10]:
criterion = nn.CrossEntropyLoss()

hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

inputs, targets = next(iter(fake_loader))
inputs = inputs.to(device)
targets = targets.to(device)

for model in models:
    model.train()
#     model.eval()
    model.zero_grad()
    loss = criterion(model(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1))
    print(loss)
    loss.backward()

print()
print("model_torch")
# print(model_torch.lstm2.weight_ih.grad[:4, :7])
print(model_torch.lstm2.beta_tanh_cell.grad[:7])
print(model_torch.lstm2.bias.grad[:25])
print("\n")
print("model_cpp")
# print(model_cpp.lstm2.weight_ih.grad[:4, :7])
print(model_cpp.lstm2.beta_tanh_cell.grad[:7])
print(model_torch.lstm2.bias.grad[:25])
print("\n")
print(model_torch.lstm2.weight_ih.grad.sub(model_cpp.lstm2.weight_ih.grad).abs().sum())

tensor(2.3148, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3148, device='cuda:0', grad_fn=<NllLossBackward>)

model_torch
tensor([ 0.0019,  0.0054,  0.0018,  0.0015,  0.0043, -0.0001,  0.0013],
       device='cuda:0')
tensor([-2.6875e-04,  1.9579e-04, -1.4493e-04, -4.1025e-04,  4.9610e-04,
        -5.6956e-05, -2.5199e-04,  5.6146e-04,  4.4483e-04,  4.4219e-04,
         1.1264e-04,  3.3381e-04,  2.7544e-04, -3.4873e-04,  8.1574e-04,
        -7.7577e-05, -9.7136e-04,  1.4679e-04,  1.6942e-04,  8.2287e-06,
        -5.0800e-04,  4.3449e-05, -3.0350e-04, -5.9347e-05, -2.0256e-04],
       device='cuda:0')


model_cpp
tensor([ 0.0019,  0.0054,  0.0018,  0.0015,  0.0043, -0.0001,  0.0013],
       device='cuda:0')
tensor([-2.6875e-04,  1.9579e-04, -1.4493e-04, -4.1025e-04,  4.9610e-04,
        -5.6956e-05, -2.5199e-04,  5.6146e-04,  4.4483e-04,  4.4219e-04,
         1.1264e-04,  3.3381e-04,  2.7544e-04, -3.4873e-04,  8.1574e-04,
        -7.7577e-05, -9.7136e-04,  1.4679e-04,  1.6942e-

---

## Forward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [11]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

model_torch.train()
model_cpp.train()

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True)

hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

In [12]:
%%timeit -n 1 -r 10
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_torch(inputs, hidden)

3.12 s ± 117 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [13]:
%%timeit -n 1 -r 10
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cpp(inputs, hidden)

7.84 s ± 186 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


---

## +Backward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [14]:
criterion = nn.CrossEntropyLoss()

In [15]:
%%timeit -r 10
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_torch.zero_grad()
    criterion(model_torch(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

9.08 s ± 119 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [16]:
%%timeit -r 10
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cpp.zero_grad()
    criterion(model_cpp(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

13.9 s ± 185 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


---